**Import intersection definition from mycity**

In the folder intersections there are files from Landau intersections

In [1]:
folder = 'input/intersections'

In [2]:
import os
import json

def find_json_files(directory):
    files = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        files.append(filepath)
    return files

In [3]:

json_files = find_json_files(folder)

**Interpret Contents of the files**

**Parse first file and understand the content**

In [4]:
import json
from collections import defaultdict

for file in json_files:

    with open(file, 'r') as f:  
        json_data = json.load(f)  

    # Extract the intersection name
    intersection_name = json_data["tlc"]["name"]

    if intersection_name == 'LD-LSA1':
    
        # Extract the detector IDs
        detector_ids = []
        print(f"### Intersection Name: {intersection_name}")

        for sw_status in json_data.get("tlc", {}).get("softwareStatus", []):
            node = sw_status.get("node", {})
            detectors = node.get("detectors", [])

            for d in detectors:
                print(
                    f"Detector ID={d['id']}, "
                    f"Name={d['name']}, "
                    f"Type={d['detectorType']}"
                )

### Intersection Name: LD-LSA1
Detector ID=7, Name=DET11, Type=NONE
Detector ID=8, Name=DET12, Type=NONE
Detector ID=9, Name=DET21, Type=NONE
Detector ID=10, Name=DET22, Type=NONE
Detector ID=11, Name=DET31, Type=NONE
Detector ID=12, Name=DET32, Type=NONE
Detector ID=13, Name=DET41, Type=NONE
Detector ID=14, Name=DET42, Type=NONE
Detector ID=15, Name=TAS1, Type=NONE
Detector ID=16, Name=TAS1a, Type=NONE
Detector ID=17, Name=TAS2, Type=NONE
Detector ID=18, Name=TAS2a, Type=NONE
Detector ID=19, Name=TAS3, Type=NONE
Detector ID=20, Name=TAS3a, Type=NONE
Detector ID=21, Name=TAS4, Type=NONE
Detector ID=22, Name=TAS4a, Type=NONE
Detector ID=23, Name=DET43, Type=NONE
Detector ID=24, Name=DET44, Type=NONE
Detector ID=25, Name=DET51, Type=NONE
Detector ID=26, Name=DET52, Type=NONE
Detector ID=27, Name=DET61, Type=NONE
Detector ID=28, Name=DET71, Type=NONE
Detector ID=29, Name=DET72, Type=NONE
Detector ID=30, Name=DET73, Type=NONE
Detector ID=31, Name=TAS5, Type=NONE
Detector ID=32, Name=TAS5a,

**Check state before using AI**

In [ ]:
import json

def check_state(tlc: dict):
    reasons = []

    # 1) Active?
    if tlc.get("active") is False:
        reasons.append("active=False")

    # 2) General State?
    state = tlc.get("state")
    if state is not None and state != "NO_FAILURE":
        reasons.append(f"state={state} (expected NO_FAILURE)")

    # 3) Is there any fault dedector producing same value or always give 0?
    broken = tlc.get("brokenDetectorCount")
    if isinstance(broken, int) and broken > 0:
        reasons.append(f"brokenDetectorCount={broken}")

    # 4) Checking the hardware status
    hw = tlc.get("hardwareStatus", {})
    if isinstance(hw, dict):
        if hw.get("powerOk") is False:
            reasons.append("hardwareStatus.powerOk=False")
        if hw.get("emergencyOff") is True:
            reasons.append("hardwareStatus.emergencyOff=True")
        if hw.get("doorOpen") is True:
            reasons.append("hardwareStatus.doorOpen=True")
        if hw.get("persistenceOk") is False:
            reasons.append("hardwareStatus.persistenceOk=False")

    # 5) Reading Flags?
    if tlc.get("readDetectors") is False:
        reasons.append("readDetectors=False")

    ok = (len(reasons) == 0)
    return ok, reasons


#  Run state check for LD-LSA1
for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        json_data = json.load(f)

    tlc = json_data.get("tlc", {})
    intersection_name = tlc.get("name")

    if intersection_name == "LD-LSA1":
        ok, reasons = check_state(tlc)

        print(f"### STATE CHECK: {intersection_name}")
        if ok:
            print("OK -> AI/analysis can proceed")
        else:
            print("NOT OK -> do NOT use AI on this snapshot")
            for r in reasons:
                print(" -", r)

        # Summary 
        print("Summary:",
              "state=", tlc.get("state"),
              "| active=", tlc.get("active"),
              "| brokenDetectorCount=", tlc.get("brokenDetectorCount"),
              "| powerOk=", tlc.get("hardwareStatus", {}).get("powerOk"),
              "| emergencyOff=", tlc.get("hardwareStatus", {}).get("emergencyOff"))
        break


### STATE CHECK: LD-LSA1
NOT OK -> do NOT use AI on this snapshot
 - state=INTERNAL_FAILURE (expected NO_FAILURE)
 - brokenDetectorCount=2
Summary: state= INTERNAL_FAILURE | active= True | brokenDetectorCount= 2 | powerOk= True | emergencyOff= False


**Which dedectors have failure?**

In [7]:
for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        j = json.load(f)
#which dedectors have failure?
    tlc = j.get("tlc", {})
    if tlc.get("name") == "LD-LSA1":
        hw = tlc.get("hardwareStatus", {})
        print("faultDetectors:", hw.get("faultDetectors"))
        break


faultDetectors: [{'detectorType': 'NONE', 'id': '29', 'name': 'DET72'}, {'detectorType': 'NONE', 'id': '25', 'name': 'DET51'}]


**Trying to apply the same code for all intersections :**

**My corridor LSA: LD-LSA16 LD-LSA10 LD-LSA8 LD-LSA24 LD-LSA29 LD-LSA32 LD-LSA3 LD-LSA11 LD-LSA5 LD-LSA4 LD-LSA40 LD-LSA42 LD-LSA1**

In [10]:
import os
import json

folder = r"C:\Users\mogul\OneDrive\Masaüstü\mt_emre\chapter 4 - build digital twin\LSTM - detector prediction\input\intersections"

def find_json_files(directory):
    files = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath) and filename.lower().endswith(".json"):
            files.append(filepath)
    return files

json_files = find_json_files(folder)

important_intersections = {
    "LD-LSA16", "LD-LSA10", "LD-LSA8", "LD-LSA24", "LD-LSA29", "LD-LSA32",
    "LD-LSA3", "LD-LSA11", "LD-LSA5", "LD-LSA4", "LD-LSA40", "LD-LSA42", "LD-LSA1"
}

found_intersections = []   # <-- buraya toplayacağız

for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        json_data = json.load(f)

    tlc = json_data.get("tlc", {})
    alias = tlc.get("alias", "")
    name = tlc.get("name", "")

    if alias in important_intersections:
        found_intersections.append(alias)

        print(f"\n### Intersection Name: {alias}")
        print(f"Full name: {name}")

        for sw_status in tlc.get("softwareStatus", []):
            node = sw_status.get("node", {})
            detectors = node.get("detectors", [])

            for d in detectors:
                print(
                    f"Detector ID={d.get('id')}, "
                    f"Name={d.get('name')}, "
                    f"Type={d.get('detectorType')}"
                )

# 🔽 EN SON: intersection ID'lerini tek satırda yazdır
print("\n=== INTERSECTIONS IN CORRIDOR ===")
print(", ".join(sorted(found_intersections)))



### Intersection Name: LD-LSA29
Full name: LD-LSA29 (Ostring/Industriestr.)
Detector ID=1, Name=D1, Type=NONE
Detector ID=2, Name=D1a, Type=NONE
Detector ID=3, Name=D2, Type=NONE
Detector ID=4, Name=D2a, Type=NONE
Detector ID=5, Name=D2b, Type=NONE
Detector ID=6, Name=D2c, Type=NONE
Detector ID=7, Name=D2d, Type=NONE
Detector ID=8, Name=D2e, Type=NONE
Detector ID=9, Name=D3, Type=NONE
Detector ID=10, Name=D3a, Type=NONE
Detector ID=11, Name=D4, Type=NONE
Detector ID=12, Name=D4a, Type=NONE
Detector ID=13, Name=D4b, Type=NONE
Detector ID=14, Name=T1, Type=NONE
Detector ID=15, Name=T3, Type=NONE
Detector ID=16, Name=BT1, Type=NONE
Detector ID=17, Name=BT2, Type=NONE
Detector ID=18, Name=BT3, Type=NONE
Detector ID=19, Name=BT4, Type=NONE
Detector ID=20, Name=FEan, Type=NONE
Detector ID=21, Name=FEab, Type=NONE
Detector ID=22, Name=VKx_iO, Type=NONE
Detector ID=23, Name=WK1_Error, Type=NONE
Detector ID=24, Name=WK3_Error, Type=NONE
Detector ID=25, Name=WK5_Error, Type=NONE
Detector ID=26,

**State Control for all intersection in SUMO**

In [11]:
import os
import json

folder = r"C:\Users\mogul\OneDrive\Masaüstü\mt_emre\chapter 4 - build digital twin\LSTM - detector prediction\input\intersections"

def find_json_files(directory):
    files = []
    for filename in os.listdir(directory):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath) and filename.lower().endswith(".json"):
            files.append(filepath)
    return files

json_files = find_json_files(folder)

important_intersections = {
    "LD-LSA16", "LD-LSA10", "LD-LSA8", "LD-LSA24", "LD-LSA29", "LD-LSA32",
    "LD-LSA3", "LD-LSA11", "LD-LSA5", "LD-LSA4", "LD-LSA40", "LD-LSA42", "LD-LSA1"
}

def state_check(tlc: dict):
    reasons = []

    # active?
    if tlc.get("active") is False:
        reasons.append("active=False")

    # state?
    state = tlc.get("state")
    if state is not None and state != "NO_FAILURE":
        reasons.append(f"state={state}")

    # broken detectors?
    broken = tlc.get("brokenDetectorCount")
    if isinstance(broken, int) and broken > 0:
        reasons.append(f"brokenDetectorCount={broken}")

    # hardwareStatus checks
    hw = tlc.get("hardwareStatus", {})
    if isinstance(hw, dict):
        if hw.get("powerOk") is False:
            reasons.append("powerOk=False")
        if hw.get("emergencyOff") is True:
            reasons.append("emergencyOff=True")
        if hw.get("doorOpen") is True:
            reasons.append("doorOpen=True")
        if hw.get("persistenceOk") is False:
            reasons.append("persistenceOk=False")

    # readDetectors?
    if tlc.get("readDetectors") is False:
        reasons.append("readDetectors=False")

    ok = (len(reasons) == 0)
    return ok, reasons


found = []
ok_list = []
not_ok_list = []

for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        j = json.load(f)

    tlc = j.get("tlc", {})
    alias = tlc.get("alias", "")

    if alias not in important_intersections:
        continue

    found.append(alias)

    ok, reasons = state_check(tlc)

    if ok:
        ok_list.append(alias)
    else:
        not_ok_list.append((alias, reasons))

# --- PRINT RESULTS ---
print("=== STATE CHECK RESULTS (Corridor Intersections) ===")
print("Total found:", len(found))
print("OK:", len(ok_list))
print("NOT OK:", len(not_ok_list))

print("\nOK intersections:")
print(", ".join(sorted(ok_list)) if ok_list else "-")

print("\nNOT OK intersections (with reasons):")
if not not_ok_list:
    print("-")
else:
    for alias, reasons in sorted(not_ok_list, key=lambda x: x[0]):
        print(f"- {alias}: " + "; ".join(reasons))

print("\n=== ALL CORRIDOR INTERSECTIONS (one line) ===")
print(", ".join(sorted(set(found))))


=== STATE CHECK RESULTS (Corridor Intersections) ===
Total found: 13
OK: 6
NOT OK: 7

OK intersections:
LD-LSA10, LD-LSA11, LD-LSA24, LD-LSA29, LD-LSA32, LD-LSA42

NOT OK intersections (with reasons):
- LD-LSA1: state=INTERNAL_FAILURE; brokenDetectorCount=2
- LD-LSA16: state=DETECTOR_FAILURE; brokenDetectorCount=2
- LD-LSA3: readDetectors=False
- LD-LSA4: state=INTERNAL_FAILURE; brokenDetectorCount=2
- LD-LSA40: state=CONNECTION_ERROR
- LD-LSA5: state=INTERNAL_FAILURE; brokenDetectorCount=2
- LD-LSA8: state=ERROR; persistenceOk=False

=== ALL CORRIDOR INTERSECTIONS (one line) ===
LD-LSA1, LD-LSA10, LD-LSA11, LD-LSA16, LD-LSA24, LD-LSA29, LD-LSA3, LD-LSA32, LD-LSA4, LD-LSA40, LD-LSA42, LD-LSA5, LD-LSA8


*Which dedectors have failure?**

In [12]:
import json

print("=== FAULT DETECTORS (Corridor) ===")

for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        j = json.load(f)

    tlc = j.get("tlc", {})
    alias = tlc.get("alias", "")

    if alias not in important_intersections:
        continue

    hw = tlc.get("hardwareStatus", {})
    faults = hw.get("faultDetectors", []) if isinstance(hw, dict) else []

    if faults:
        names = [d.get("name") for d in faults if isinstance(d, dict)]
        print(f"- {alias}: {', '.join([n for n in names if n])}")
    else:
        print(f"- {alias}: (no faultDetectors listed)")


=== FAULT DETECTORS (Corridor) ===
- LD-LSA29: (no faultDetectors listed)
- LD-LSA3: (no faultDetectors listed)
- LD-LSA5: WBK2_Err, WBK1_Err
- LD-LSA16: WD41, WD42
- LD-LSA10: (no faultDetectors listed)
- LD-LSA42: (no faultDetectors listed)
- LD-LSA40: (no faultDetectors listed)
- LD-LSA11: (no faultDetectors listed)
- LD-LSA24: (no faultDetectors listed)
- LD-LSA32: (no faultDetectors listed)
- LD-LSA1: DET72, DET51
- LD-LSA8: (no faultDetectors listed)
- LD-LSA4: DET2_2, DET2_4


**For LSTM Training**

In [14]:
import json

def state_check_and_faults(tlc: dict):
    reasons = []

    # state check
    if tlc.get("active") is False:
        reasons.append("active=False")

    state = tlc.get("state")
    if state is not None and state != "NO_FAILURE":
        reasons.append(f"state={state}")

    broken = tlc.get("brokenDetectorCount")
    if isinstance(broken, int) and broken > 0:
        reasons.append(f"brokenDetectorCount={broken}")

    hw = tlc.get("hardwareStatus", {})
    if isinstance(hw, dict):
        if hw.get("powerOk") is False:
            reasons.append("powerOk=False")
        if hw.get("emergencyOff") is True:
            reasons.append("emergencyOff=True")
        if hw.get("doorOpen") is True:
            reasons.append("doorOpen=True")
        if hw.get("persistenceOk") is False:
            reasons.append("persistenceOk=False")

        fault_list = hw.get("faultDetectors", [])
    else:
        fault_list = []

    fault_names = [
        d.get("name") for d in fault_list
        if isinstance(d, dict) and d.get("name")
    ]

    ok = (len(reasons) == 0)
    return ok, reasons, fault_names


# --- Build notes for each corridor intersection ---
notes = []

for file in json_files:
    with open(file, "r", encoding="utf-8") as f:
        j = json.load(f)

    tlc = j.get("tlc", {})
    alias = tlc.get("alias", "")

    if alias not in important_intersections:
        continue

    ok, reasons, fault_names = state_check_and_faults(tlc)

    # Decision logic (clear and defensible for a thesis)
    decision = "USE for LSTM training" if ok else "DO NOT use for LSTM training"

    cleaning_note = (
        "No detector cleaning required"
        if not fault_names
        else "Mask / remove faulty detectors: " + ", ".join(fault_names)
    )

    reason_note = (
        "Reason: operational state is normal (NO_FAILURE, no broken detectors)"
        if ok
        else "Reason: " + "; ".join(reasons)
    )

    notes.append((alias, decision, cleaning_note, reason_note))


# --- Print final notes (one by one) ---
print("=== INTERSECTION USABILITY NOTES FOR LSTM ===\n")

for alias, decision, cleaning_note, reason_note in sorted(notes, key=lambda x: x[0]):
    print(f"- {alias}: {decision}")
    print(f"  {cleaning_note}")
    print(f"  {reason_note}\n")


=== INTERSECTION USABILITY NOTES FOR LSTM ===

- LD-LSA1: DO NOT use for LSTM training
  Mask / remove faulty detectors: DET72, DET51
  Reason: state=INTERNAL_FAILURE; brokenDetectorCount=2

- LD-LSA10: USE for LSTM training
  No detector cleaning required
  Reason: operational state is normal (NO_FAILURE, no broken detectors)

- LD-LSA11: USE for LSTM training
  No detector cleaning required
  Reason: operational state is normal (NO_FAILURE, no broken detectors)

- LD-LSA16: DO NOT use for LSTM training
  Mask / remove faulty detectors: WD41, WD42
  Reason: state=DETECTOR_FAILURE; brokenDetectorCount=2

- LD-LSA24: USE for LSTM training
  No detector cleaning required
  Reason: operational state is normal (NO_FAILURE, no broken detectors)

- LD-LSA29: USE for LSTM training
  No detector cleaning required
  Reason: operational state is normal (NO_FAILURE, no broken detectors)

- LD-LSA3: USE for LSTM training
  No detector cleaning required
  Reason: operational state is normal (NO_FAI

In [17]:
# Only physical vehicle detectors (DET) are going to be used???
# as inputs to the LSTM model.
# Internal controller signals, timers,
# bus priority triggers, and fault indicators
# (e.g., T*, BT*, VK*, FE*)
# are going to excluded???, as they do not represent
# traffic flow measurements.???
